this takes in the raw E-Prime files, which have been concatonated together into one large file (raw_concat.csv), and does minimal processing 

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time 

In [ ]:
#this makes it so that the WHOLE dataframe prints, just fyi, this can bog down your code

pd.set_option('display.max_columns', 1000)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', 199)  # or 199
pd.options.display.max_seq_items = 2000


In [ ]:
drop_cols = ['Unnamed: 0', 'Allowed', 'Subject', 'StudioVersion', 
              'triggercode', 'Clock.Information', 
             'DataFile.Basename', 'Display.RefreshRate', 'RandomSeed', '-146510705', '0', '0-SST_GE_20161208_TWoptimized_Consolidated', 
            '03-11-2017', '09-29-2018', '1', '1.0.0.721',
       '1.0.0.729', '1.1', 'VersionPersist: 1', 'leftarrow', 'rightarrow',
       'triggercode', 'v11Proc', 'v3ProcList', 'v3ProcList.Cycle',
       'v3ProcList.Sample', '{^}57{=}', 'ConsecSameResp', 'ControlAcc', 'CorrectAnswer', 'CorrectResponse',
       'Cue2Back.Duration', 'Cue2Back.DurationError', 'Cue2Back.FinishTime',
       'Cue2Back.OffsetDelay', 'Cue2Back.OffsetTime', 'Cue2Back.OnsetDelay',
       'Cue2Back.OnsetTime', 'Cue2Back.OnsetToOnsetTime', 'Cue2Back.StartTime',
       'CueFix.Duration', 'CueFix.DurationError', 'CueFix.FinishTime',
       'CueFix.OffsetDelay', 'CueFix.OffsetTime', 'CueFix.OnsetDelay',
       'CueFix.OnsetTime', 'CueFix.OnsetToOnsetTime', 'CueFix.RTTime',
       'CueFix.StartTime', 'CueTarget.Duration', 'CueTarget.DurationError',
       'CueTarget.FinishTime', 'CueTarget.OffsetDelay', 'CueTarget.OffsetTime',
       'CueTarget.OnsetDelay', 'CueTarget.OnsetTime',
       'CueTarget.OnsetToOnsetTime', 'CueTarget.StartTime', 'GetReady.OffsetTime', 'GetReady.OnsetDelay', 'GetReady.OnsetTime',
       'GetReady.OnsetToOnsetTime', 'GetReady.RTTime',
       'GetReady2.DurationError', 'GetReady2.OffsetDelay',
       'GetReady2.OffsetTime', 'GetReady2.OnsetDelay', 'GetReady2.OnsetTime',
       'GetReady2.OnsetToOnsetTime', 'GetReady2.RTTime',  'SiemensPad.FinishTime', 'SiemensPad.OffsetDelay',
       'SiemensPad.OffsetTime', 'SiemensPad.OnsetDelay',
       'SiemensPad.OnsetTime', 'SiemensPad.OnsetToOnsetTime',
       'SiemensPad.RTTime', 'Stim.ACC', 'Stim.CRESP', 'Stim.Duration',
       'Stim.DurationError', 'Stim.FinishTime', 'Stim.OffsetDelay',
       'Stim.OffsetTime', 'Stim.OnsetDelay', 'Stim.OnsetTime',
       'Stim.OnsetToOnsetTime', 'Stim.RESP', 'Stim.RT', 'Stim.RTTime',
       'Stim.StartTime', '0-SST_SP_20161208', '1.10', '1.11', '1.12', '1.2', '1.3', 
       '1.4', '1.5', '1.6', '1.7', '1.8', '1.9', '104234', '10:10:09', '11', '12', 
       '17:05:53', '172646',  '2', '2.0.10.252', '2.0.10.356', '2.0.10.356.1','270384696',
       '3', '59.791', '60.007', '9/29/2018 3:10:09 PM', '<?xml version="1.0"?>\\n<Clock xmlns:dt="urn:schemas-microsoft-com:datatypes"><Description dt:dt="string">E-Prime Primary Realtime Clock</Description><StartTime><Timestamp dt:dt="int">0</Timestamp><DateUtc dt:dt="string">2017-03-11T23:05:52Z</DateUtc></StartTime><FrequencyChanges><FrequencyChange><Frequency dt:dt="r8">2143505</Frequency><Timestamp dt:dt="r8">1670595342820</Timestamp><Current dt:dt="r8">0</Current><DateUtc dt:dt="string">2017-03-11T23:05:52Z</DateUtc></FrequencyChange></FrequencyChanges></Clock>\\n','<?xml version="1.0"?>\\n<Clock xmlns:dt="urn:schemas-microsoft-com:datatypes"><Description dt:dt="string">E-Prime Primary Realtime Clock</Description><StartTime><Timestamp dt:dt="int">0</Timestamp><DateUtc dt:dt="string">2018-09-29T15:10:09Z</DateUtc></StartTime><FrequencyChanges><FrequencyChange><Frequency dt:dt="r8">3328369</Frequency><Timestamp dt:dt="r8">3200020804564</Timestamp><Current dt:dt="r8">0</Current><DateUtc dt:dt="string">2018-09-29T15:10:09Z</DateUtc></FrequencyChange></FrequencyChanges></Clock>\\n', 'SST_GE_20161208_TWoptimized_Consolidated-61RKCL4Y-1-11',  
        'SST_SP_20161208-5YKVY4EN-1-11', '34', '4',  '5YKVY4EN', '61RKCL4Y', 'FontStyle'
       ]

In [ ]:
raw_concat = pd.read_csv('raw_concat_jan14.csv', low_memory=False).drop(drop_cols, axis=1)

In [ ]:
raw_concat = raw_concat[np.logical_and(raw_concat.TrialCode != 'BeginFix', \
                                       raw_concat.TrialCode != 'EndFix')]

raw_concat = raw_concat[raw_concat['TrialCode'].notnull()]

raw_concat['StopTrial'] = pd.Series() 
raw_concat['TrialNum'] = pd.Series() 
incomplete_subs = []

In [ ]:
counter = 0
print(counter)
for i in raw_concat['NARGUID'].unique(): 
    counter +=1
    if counter % 100 == 0:
        print(counter)
    sub_idx = raw_concat['NARGUID'] == i 
    raw_concat['TrialNum'].loc[sub_idx] = np.arange(1, len(raw_concat.loc[sub_idx])+1)
    if raw_concat['TrialNum'].loc[sub_idx].max() != 360:
        raw_concat.drop(index=sub_idx[sub_idx == True].index)
        incomplete_subs.append(i)
        print(i, 'dropped')
    else:   
        stop_trial_mask = (raw_concat['TrialCode'].loc[sub_idx] ==      
                           'IncorrectStop') | \
                        (raw_concat['TrialCode'].loc[sub_idx] == 
                            'CorrectStop')

        stop_trial_idx = stop_trial_mask[stop_trial_mask == True].index
        raw_concat['StopTrial'] = np.NaN
        raw_concat['StopTrial'].loc[stop_trial_idx] = np.arange(1, len(raw_concat.loc[stop_trial_idx])+ 1)
        end_time = time.time()

In [ ]:
#this saves as a csv so you can avoid running that loops multiple times 
date_string  = datetime.date.today().strftime('%m') + '_' + datetime.date.today().strftime('%d')
SST_concat.to_csv(f'partially_cleaned_{date_string}.csv')

In [ ]:
def switch(x):
    """
    switch responses for participants who had 
    flipped response paddles
    """
    if x == 1:
        return 2
    elif x == 2:
        return 1
    elif x == 3:
        return 4
    elif x == 4:
        return 3
    else:
        return np.nan

In [ ]:
#find subjects who require this 
switch_subjects = pd.read_csv('../../ABCDStudyNDA/abcd_sst02.txt', sep = '\t', index_col='subjectkey', low_memory=False)
switch_subjects = switch_subjects['tfmri_sst_beh_switchflag'].astype(str)
switch_subjects = switch_subjects[switch_subjects == '1'].index
switch_subjects = [i[-8:] for i in switch_subjects]

In [ ]:
resp_cols = ['Go.RESP', 'Fix.RESP', 'StopSignal.RESP', 'SSD.RESP']

for sub in switch_subjects: 
    sub_index = SST_concat.loc[SST_concat['NARGUID'] == sub].index 
    for col in resp_cols:
        SST_concat.loc[sub_index, col] = SST_concat.loc[sub_index, col].apply(switch)
        

In [ ]:
### Check "TrialCode" accuracy

In [ ]:
#some subjects have their subtrial under Procedure[Trial]
SST_concat['Procedure[SubTrial]'].loc[SST_concat['Procedure[SubTrial]'].isnull()] = SST_concat['Procedure[Trial]'] 
#the trial type column has inconsistent notation for stop trials 
SST_concat['trial_type'] = SST_concat['Procedure[SubTrial]'].replace('VariableStopTrial.*', 'StopTrial', regex=True)

In [ ]:
#find all go trials 
go_trial_mask = SST_concat.loc[SST_concat['trial_type'] == 'GoTrial']
go_trial_idx = go_trial_mask[go_trial_mask==True].index

In [ ]:
#the response recordings are inconsistent in type (str, int, float) this fixes that 

cresp_replace = {'2.0': 2.0,
                 '1.0': 1.0,
                 '3.0': 3.0,
                 '4.0': 4.0,
                 '1,{LEFTARROW}': 1.0,
                 '2,{RIGHTARROW}': 2.0}

resp_replace = {'2.0': 2.0,
                '1.0': 1.0,
                '3.0': 3.0,
                '4.0': 4.0,
                '{LEFTARROW}': 1.0,
                '{RIGHTARROW}': 2.0}

SST_concat['Go.CRESP'].replace(to_replace = cresp_replace, inplace=True)

SST_concat['Go.RESP'].replace(to_replace = resp_replace, inplace = True)
SST_concat['Fix.RESP'].replace(to_replace = resp_replace, inplace=True)
SST_concat['SSD.RESP'].replace(to_replace = resp_replace, inplace=True)
SST_concat['StopSignal.RESP'].replace(to_replace = resp_replace, inplace=True)

In [ ]:
#create my own correct response column 
SST_concat['correct_go_response'] = np.NaN

SST_concat['correct_go_response'].loc[(~SST_concat['Go.RESP'].isnull()) & 
                                      (SST_concat['Go.CRESP'] == SST_concat['Go.RESP'])] = float(1)

SST_concat['correct_go_response'].loc[(SST_concat['Go.RESP'].isnull()) & 
                                      (SST_concat['Go.CRESP'] == SST_concat['Fix.RESP'])] = float(1)

SST_concat['correct_go_response'].loc[(~SST_concat['Go.RESP'].isnull()) & 
                                      (SST_concat['Go.CRESP'] != SST_concat['Go.RESP']) &
                                      (SST_concat['trial_type'] == 'GoTrial')] = float(0)


SST_concat['correct_go_response'].loc[(SST_concat['Go.RESP'].isnull()) & 
                                      (SST_concat['Go.CRESP'] != SST_concat['Fix.RESP']) &
                                      (SST_concat['trial_type'] == 'GoTrial')] = float(0)


SST_concat['correct_go_response'].loc[(SST_concat['Go.RESP'].isnull()) & (SST_concat['Fix.RESP'].isnull()) & 
                                      (SST_concat['trial_type'] == 'GoTrial')] = 'omission'




In [ ]:
#check that I only have ones, zeros, and omissions
SST_concat.loc[SST_concat['trial_type'] == 'GoTrial']['correct_go_response'].unique() 

In [ ]:
#correct stop column 
SST_concat['correct_stop'] = np.NaN

SST_concat['correct_stop'].loc[(SST_concat['StopSignal.RESP'].isnull()) & (SST_concat['Fix.RESP'].isnull()) & \
                               (SST_concat['trial_type'] == 'StopTrial') & (SST_concat['SSD.RESP'].isnull())] = float(1)

SST_concat['correct_stop'].loc[(~(SST_concat['StopSignal.RESP'].isnull()) | ~(SST_concat['Fix.RESP'].isnull()) \
                                | ~(SST_concat['SSD.RESP'].isnull())) & (SST_concat['trial_type'] == 'StopTrial')] = float(0)

In [ ]:
SST_concat['correct_stimulus_mapping_1'] = np.NaN
SST_concat['correct_stimulus_mapping_2'] = np.NaN

In [ ]:
SST_concat.loc[SST_concat['Stimulus'] == 'images/Right_Arrow.bmp', 'correct_stimulus_mapping_1'] = SST_concat.loc[SST_concat['Stimulus'] == 'images/Right_Arrow.bmp']['Go.CRESP'].dropna().unique()[0]
SST_concat.loc[SST_concat['Stimulus'] == 'images/Right_Arrow.bmp', 'correct_stimulus_mapping_2'] = SST_concat.loc[SST_concat['Stimulus'] == 'images/Right_Arrow.bmp']['Go.CRESP'].dropna().unique()[1]

SST_concat.loc[SST_concat['Stimulus'] == 'images/Left_Arrow.bmp', 'correct_stimulus_mapping_1'] = SST_concat.loc[SST_concat['Stimulus'] == 'images/Left_Arrow.bmp']['Go.CRESP'].dropna().unique()[0]
SST_concat.loc[SST_concat['Stimulus'] == 'images/Left_Arrow.bmp', 'correct_stimulus_mapping_2'] = SST_concat.loc[SST_concat['Stimulus'] == 'images/Left_Arrow.bmp']['Go.CRESP'].dropna().unique()[1]

In [ ]:
#correct stop choice response 
SST_concat['correct_stop_mapping'] = np.NaN 
SST_concat['correct_stop_mapping'].loc[(SST_concat['correct_stop'] == 0) & (~SST_concat['SSD.RESP'].isnull()) &\
                                       ((SST_concat['SSD.RESP'] == SST_concat['correct_stimulus_mapping_1']) | (SST_concat['SSD.RESP'] == SST_concat['correct_stimulus_mapping_2']))] = float(1)

SST_concat['correct_stop_mapping'].loc[(SST_concat['correct_stop'] == 0) & (~SST_concat['SSD.RESP'].isnull()) &\
                                      (SST_concat['SSD.RESP'] != SST_concat['correct_stimulus_mapping_1']) & (SST_concat['SSD.RESP'] != SST_concat['correct_stimulus_mapping_2'])] = float(0)


In [ ]:
SST_concat['correct_stop_mapping'].loc[(SST_concat['correct_stop'] == 0) & (SST_concat['SSD.RESP'].isnull()) & (~SST_concat['StopSignal.RESP'].isnull()) &\
                                       ((SST_concat['StopSignal.RESP'] == SST_concat['correct_stimulus_mapping_1']) | (SST_concat['StopSignal.RESP'] == SST_concat['correct_stimulus_mapping_2']))] = float(1)

SST_concat['correct_stop_mapping'].loc[(SST_concat['correct_stop'] == 0) & (SST_concat['SSD.RESP'].isnull()) & (~SST_concat['StopSignal.RESP'].isnull()) &\
                                       ((SST_concat['StopSignal.RESP'] != SST_concat['correct_stimulus_mapping_1']) & (SST_concat['StopSignal.RESP'] != SST_concat['correct_stimulus_mapping_2']))] = float(0)

In [ ]:
SST_concat['correct_stop_mapping'].loc[(SST_concat['correct_stop'] == 0) & (SST_concat['SSD.RESP'].isnull()) & (SST_concat['StopSignal.RESP'].isnull()) & (~SST_concat['Fix.RESP'].isnull()) &\
                                       ((SST_concat['Fix.RESP'] == SST_concat['correct_stimulus_mapping_1']) | (SST_concat['Fix.RESP'] == SST_concat['correct_stimulus_mapping_2']))] = float(1)

SST_concat['correct_stop_mapping'].loc[(SST_concat['correct_stop'] == 0) & (SST_concat['SSD.RESP'].isnull()) & (SST_concat['StopSignal.RESP'].isnull()) & (~SST_concat['Fix.RESP'].isnull()) &\
                                       ((SST_concat['Fix.RESP'] != SST_concat['correct_stimulus_mapping_1']) & (SST_concat['Fix.RESP'] != SST_concat['correct_stimulus_mapping_2']))] = float(0)

### fix the go.rt calculation

since the trial unfolds over several different sub trials, it's necessary to add some of these together to get correct rt

In [ ]:
#find the index of where there's a fixation response on go trials 
go_fix_resp = (~SST_concat['Fix.RESP'].isnull()) & \
              (SST_concat['Go.RESP'].isnull()) & \
              (SST_concat['trial_type'] == 'GoTrial')
go_fix_idx = go_fix_resp[go_fix_resp == True].index

In [ ]:
#make column from Go.RT values
SST_concat['go_rt_adjusted'] = SST_concat['Go.RT'].copy()
#for long response trials, add the fix.rt to the go.duration
SST_concat['go_rt_adjusted'][go_fix_idx] = SST_concat.loc[go_fix_idx]['Go.Duration'] +  \
                                           SST_concat.loc[go_fix_idx]['Fix.RT']

#### fix the stop sig rt calculation

In [ ]:
#find all stop trials
stop_trial_mask = (SST_concat['trial_type'] == 'StopTrial')
#make an index of all stop trials 
stop_trial_idx = stop_trial_mask[stop_trial_mask == True].index

In [ ]:
SST_concat['stop_rt_adjusted'] = SST_concat['StopSignal.RT']
#find all stop failure trials with resp during fix.resp, and no resp during stopsignal.resp
stop_fix_resp = SST_concat['Fix.RESP'].notnull() & \
                SST_concat['StopSignal.RESP'].isnull() & \
                ((SST_concat['trial_type'] == 'StopTrial') & (SST_concat['correct_stop'] == 0))

stop_fix_idx = stop_fix_resp[stop_fix_resp == True].index

stop_SSD_resp = SST_concat['SSD.RESP'].notnull() & (SST_concat['trial_type'] == 'StopTrial')

In [ ]:
#use that index to add stop signal duration and stop signal duration to get correct stop signal RT on these trials
SST_concat['stop_rt_adjusted'][stop_fix_resp] = SST_concat.loc[stop_fix_resp]['StopSignal.Duration'] +\
                                           SST_concat.loc[stop_fix_resp]['Fix.RT'] + \
                                         SST_concat[stop_fix_resp]['SSDDur']

SST_concat['stop_rt_adjusted'][stop_SSD_resp] = SST_concat.loc[stop_SSD_resp]['SSD.RT']

In [ ]:
#find the stop failure trials and add the stop signal duration for correct stop fail RT 

stop_resp_mask = ((SST_concat['StopSignal.RESP'].notnull()  \
                & SST_concat['SSD.RESP'].isnull()) & \
                 (SST_concat['trial_type'] == 'StopTrial'))
        
stop_resp_idx = stop_resp_mask[stop_resp_mask == True].index
#add SSDDur to response time 
SST_concat['stop_rt_adjusted'][stop_resp_idx] = \
        SST_concat['stop_rt_adjusted'][stop_resp_idx] + SST_concat['SSDDur'][stop_resp_idx]

In [ ]:
#create column for Go.Stim.Duration - when there's no go response, this should be 1000ms
SST_concat['go_stim_duration'] = SST_concat['Go.RT'].copy()
SST_concat['go_stim_duration'].loc[SST_concat['Go.RT'] == 0] = SST_concat['Go.Duration'].loc[SST_concat['Go.RT'] == 0]

SST_concat['go_stim_duration'].loc[~(SST_concat['SSD.RESP'].isnull())] = SST_concat.loc[~SST_concat['SSD.RESP'].isnull()]['SSD.RT']
SST_concat['go_stim_duration'].loc[(SST_concat['SSD.RT']  == 0)] = SST_concat.loc[SST_concat['SSD.RT'] == 0]['SSDDur']


In [ ]:
#this saves it to csvs that are small enough to be viewed by excel
#SST_concat.iloc[:867600].to_csv('SST_cleaned_7231_1.csv')
#SST_concat.iloc[867600:1735560, :].to_csv('SST_cleaned_7231_2.csv')
#SST_concat.iloc[1735560:].to_csv('SST_cleaned_7231_3.csv')

In [ ]:
SST_concat.to_csv(f'SST_cleaned_8464_all_rows_all_columns_switch_jan19.csv')